In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from pandas import json_normalize

## Data Acquisition

In [ ]:
with open('nyc_geo.json') as f:
    nyc_geo_json = json.load(f)

#### Exploring the NYC Json geo DataFrame

In [ ]:
nyc_geo_json['features'][0] # Parse into the 1st index in features

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [ ]:
features = json_normalize(nyc_geo_json, record_path ='features') # Normalize json features, create df
features.head()

,type,id,geometry_name,geometry.type,geometry.coordinates,properties.name,properties.stacked,properties.annoline1,properties.annoline2,properties.annoline3,properties.annoangle,properties.borough,properties.bbox
0,Feature,nyu_2451_34572.1,geom,Point,"[-73.84720052054902, 40.89470517661]",Wakefield,1,Wakefield,None,None,0.0,Bronx,"[-73.84720052054902, 40.89470517661, -73.84720..."
1,Feature,nyu_2451_34572.2,geom,Point,"[-73.82993910812398, 40.87429419303012]",Co-op City,2,Co-op,City,None,0.0,Bronx,"[-73.82993910812398, 40.87429419303012, -73.82..."
2,Feature,nyu_2451_34572.3,geom,Point,"[-73.82780644716412, 40.887555677350775]",Eastchester,1,Eastchester,None,None,0.0,Bronx,"[-73.82780644716412, 40.887555677350775, -73.8..."
3,Feature,nyu_2451_34572.4,geom,Point,"[-73.90564259591682, 40.89543742690383]",Fieldston,1,Fieldston,None,None,0.0,Bronx,"[-73.90564259591682, 40.89543742690383, -73.90..."
4,Feature,nyu_2451_34572.5,geom,Point,"[-73.9125854610857, 40.890834493891305]",Riverdale,1,Riverdale,None,None,0.0,Bronx,"[-73.9125854610857, 40.890834493891305, -73.91..."


In [ ]:
# Compare similarly styled features

features['properties.name'].value_counts(), features['properties.annoline1'].value_counts()

(Bay Terrace      2
 Murray Hill      2
 Chelsea          2
 Sunnyside        2
 Wakefield        1
                 ..
 Fort Hamilton    1
 Ocean Parkway    1
 South Side       1
 North Side       1
 Fox Hills        1
 Name: properties.name, Length: 302, dtype: int64,
 East                5
 New                 5
 Sunnyside           3
 Jamaica             3
 West                3
                    ..
 Central             1
 Yorkville           1
 Lenox               1
 Roosevelt Island    1
 Fox                 1
 Name: properties.annoline1, Length: 263, dtype: int64)

#### We can see above that the feature 'properties.name' contains less duplicates and more unique variables.

In [ ]:
# Compare similarly styled features

features['geometry.coordinates'].value_counts(), features['properties.bbox'].value_counts()

([-73.84720052054902, 40.89470517661]        1
 [-74.1071817826561, 40.63187892654607]      1
 [-74.11918058534842, 40.61333593766742]     1
 [-74.17464532993542, 40.63968297845542]     1
 [-74.15008537046981, 40.632546390481124]    1
                                            ..
 [-73.93690027985234, 40.85190252555305]     1
 [-73.99427936255978, 40.71561842231432]     1
 [-74.03197914537984, 40.61476812694226]     1
 [-73.96836678035541, 40.61305976667942]     1
 [-74.08173992211962, 40.61731079252983]     1
 Name: geometry.coordinates, Length: 306, dtype: int64,
 [-73.84720052054902, 40.89470517661, -73.84720052054902, 40.89470517661]            1
 [-74.1071817826561, 40.63187892654607, -74.1071817826561, 40.63187892654607]        1
 [-74.11918058534842, 40.61333593766742, -74.11918058534842, 40.61333593766742]      1
 [-74.17464532993542, 40.63968297845542, -74.17464532993542, 40.63968297845542]      1
 [-74.15008537046981, 40.632546390481124, -74.15008537046981, 40.63254639048112

#### We can see above that the feature 'geometry.coordinates' contains the same latitude and longitude coordinates as bbox without the doubling.

## Data Organization
#### Parse the json nyc_geo.json into the dataframe with the following columns:

- Borough
- Neighborhood
- Latitude
- Longitude

In [ ]:
# Create dataframe out of latitude and longitude of neighborhoods

LatLon = pd.DataFrame(features['geometry.coordinates'].to_list(), columns=['Longitude', 'Latitude'])

# Create dataframe with neighborhood names and boroughs

df1=features[['properties.name', 'properties.borough']]

dataFrames=[df1, LatLon] # Create variable containing a list of both dataframes

nyc_df = pd.concat(dataFrames, axis=1) # Concatenate both dataframes

# Rename columns to best reflect their associated features
nyc_df = nyc_df.rename(columns={'properties.name' : 'Neighborhood', 'properties.borough': 'Borough'})

nyc_df.head()

,Neighborhood,Borough,Longitude,Latitude
0,Wakefield,Bronx,-73.847201,40.894705
1,Co-op City,Bronx,-73.829939,40.874294
2,Eastchester,Bronx,-73.827806,40.887556
3,Fieldston,Bronx,-73.905643,40.895437
4,Riverdale,Bronx,-73.912585,40.890834


In [ ]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
    len(nyc_df['Borough'].unique()),
    nyc_df.shape[0]))

The dataframe has 5 boroughs and 306 neighborhoods.


### Dealing with Duplicates

- The following neighborhoods share a name with another neighborhood in a separate borough.

In [ ]:
nyc_doubles = nyc_df['Neighborhood'].value_counts() # Get value counts of filtered neighborhood column
nyc_doubles[nyc_doubles>1] # Show which neighborhoods appear more than once

Bay Terrace    2
Murray Hill    2
Chelsea        2
Sunnyside      2
Name: Neighborhood, dtype: int64

In [ ]:
nyc_df[nyc_df['Neighborhood'] == 'Chelsea'] # View one of the doubled neighborhoods

,Neighborhood,Borough,Longitude,Latitude
116,Chelsea,Manhattan,-74.003116,40.744035
244,Chelsea,Staten Island,-74.189560,40.594726


##### We will parse through the dataframe to place the borough name at the end of the neighborhood name so as to distinguish between them.

In [ ]:
for i in range(nyc_df.shape[0]): # Create for loop
    nyc_in = nyc_df.loc[i, 'Neighborhood'] # Set variable to get a list of neighborhood names
    if nyc_df[nyc_df['Neighborhood'] == nyc_in].shape[0] > 1: # Conditional where neighborhood shows up twice
        nyc_in1 = nyc_df[nyc_df['Neighborhood'] == nyc_in].index.tolist() # append to list if condition is met
        for j in nyc_in1: # Nested for loop to go through boroughs of doubled neighborhoods
            nyc_in2 = nyc_df.loc[j, 'Borough'] # Variable set to be list of boroughs of appended neighborhoods
            nyc_df.loc[j, 'Neighborhood'] = nyc_in + ', ' + nyc_in2 # iterate through original list to add borough name separated by comma to doubled neighborhood

In [ ]:
nyc_df[nyc_df['Neighborhood'].str.startswith('Chelsea')] # View doubled neighborhood to see results

,Neighborhood,Borough,Longitude,Latitude
116,"Chelsea, Manhattan",Manhattan,-74.003116,40.744035
244,"Chelsea, Staten Island",Staten Island,-74.189560,40.594726


### Check for missing values

In [ ]:
nyc_df.isnull().sum()

Neighborhood    0
Borough         0
Longitude       0
Latitude        0
dtype: int64

## Cleaning and Joining Healthcare Facility Data with Neighborhood Data

In [ ]:
df_health = pd.read_csv('nyc_healthcare.csv') # Import nyc_healthcare dataset

In [ ]:
nyc_df1 = nyc_df.copy() # Create copy of dataframe so as to not impact original
nyc_df1.head()

,Neighborhood,Borough,Longitude,Latitude
0,Wakefield,Bronx,-73.847201,40.894705
1,Co-op City,Bronx,-73.829939,40.874294
2,Eastchester,Bronx,-73.827806,40.887556
3,Fieldston,Bronx,-73.905643,40.895437
4,Riverdale,Bronx,-73.912585,40.890834


#### Find and clean differences between the dataframes

In [ ]:
uniq_neighborhoods = df_health1['Neighborhood'].unique() # Get the unique neighborhoods from healthcare df
uniq_nyc = nyc_df['Neighborhood'].unique() # Get the unique neighborhoods from the city df
uniq_diffs = list(set(uniq_neighborhoods).difference(uniq_nyc)) # Get the non-matching neighborhoods from both.
similarities = list(set(uniq_neighborhoods).intersection(uniq_nyc)) # Get matching neighborhoods
uniq_rev_dif = list(set(uniq_nyc).difference(uniq_neighborhoods))
print(len(similarities))
print(len(uniq_diffs))
print(len(uniq_rev_dif))

In [ ]:
uniq_differences = uniq_diffs + uniq_rev_dif # Concatenate unique lists
uniq_differences = list(dict.fromkeys(uniq_differences)) # Get rid of duplicates
uniq_differences = [x for x in uniq_differences if str(x) != 'nan'] # Get rid of NaN values
uniq_difference = sorted(uniq_differences) # Sort values alphabetically
uniq_difference = pd.DataFrame(uniq_difference)
uniq_difference = uniq_difference.rename(columns={0: 'Unique Neighborhoods'})
uniq_difference1 = uniq_difference.head(10)
uniq_difference2 = uniq_difference[10:20]
uniq_difference2

,Unique Neighborhoods
10,Bedford Park
11,Bedford Stuyvesant
12,Beekman
13,Belle Harbor
14,Bellerose
15,Bellevue
16,Bergen Beach
17,Beverly Square West
18,Bloomfield
19,BoCoCa


In [ ]:
# declare an empty list to store
# latitude and longitude of values 
# of city column
longitude = []
latitude = []
   
# function to find the coordinate
# of a given city 
def findGeocode(city):
       
    # try and catch is used to overcome
    # the exception thrown by geolocator
    # using geocodertimedout  
    try:
          
        # Specify the user_agent as your
        # app name it should not be none
        geolocator = Nominatim(user_agent="Jules098")
          
        return geolocator.geocode(city)
      
    except GeocoderTimedOut:
          
        return findGeocode(city)    
  
# each value from city column
# will be fetched and sent to
# function find_geocode   
for i in (uniq_difference1["Unique Neighborhoods"]):
      
    if findGeocode(i) != None:
           
        loc = findGeocode(i)
          
        # coordinates returned from 
        # function is stored into
        # two separate list
        latitude.append(loc.latitude)
        longitude.append(loc.longitude)
       
    # if coordinate for a city not
    # found, insert "NaN" indicating 
    # missing value 
    else:
        latitude.append(np.nan)
        longitude.append(np.nan)

In [ ]:
latitude, longitude

([41.7690519,
  40.6834364,
  41.6109269,
  40.5767844,
  40.7242205,
  47.6144219,
  40.6203822,
  42.519116,
  40.7510398,
  50.0697456],
 [-87.7770096,
  -73.9412488,
  -73.7129076,
  -73.84782868947303,
  -73.7164845,
  -122.192337,
  -73.9068033,
  -83.227226,
  -92.4143872,
  19.9260176])

## Data Cleaning

In [ ]:
print(df_health.shape) # Display shape of healthcare dataframe
df_health.head()

(6069, 6)


,Venue,Category,Latitude,Longitude,Neighborhood,ZIP
0,Wakefield Eye Care,Optometrist,40.898682,-73.849352,Wakefield,10466
1,Advanced Dental Group,Dentist,40.875278,-73.828492,Co-op City,10475
2,Creating Smiles,Dentist,40.871558,-73.830285,Co-op City,10475
3,NYC Health & Hospitals,Medical Center,40.873701,-73.826771,Co-op City,10475
4,Montefiore Medical Group,Physician,40.877116,-73.828470,Co-op City,10475


In [ ]:
nyc_df.head()

,Neighborhood,Borough,Longitude,Latitude
0,Wakefield,Bronx,-73.847201,40.894705
1,Co-op City,Bronx,-73.829939,40.874294
2,Eastchester,Bronx,-73.827806,40.887556
3,Fieldston,Bronx,-73.905643,40.895437
4,Riverdale,Bronx,-73.912585,40.890834


In [ ]:
# Compare number of unique neighborhoods between the two datasets

df_health['Neighborhood'].nunique(), nyc_df['Neighborhood'].nunique()

(241, 306)

In [ ]:
health_merged = df_health.merge(nyc_df, how='inner', on='Neighborhood') # Merge datasets on Neighborhood
health_merged.head()

,Venue,Category,Latitude_x,Longitude_x,Neighborhood,ZIP,Borough,Longitude_y,Latitude_y
0,Wakefield Eye Care,Optometrist,40.898682,-73.849352,Wakefield,10466,Bronx,-73.847201,40.894705
1,Advanced Dental Group,Dentist,40.875278,-73.828492,Co-op City,10475,Bronx,-73.829939,40.874294
2,Creating Smiles,Dentist,40.871558,-73.830285,Co-op City,10475,Bronx,-73.829939,40.874294
3,NYC Health & Hospitals,Medical Center,40.873701,-73.826771,Co-op City,10475,Bronx,-73.829939,40.874294
4,Montefiore Medical Group,Physician,40.877116,-73.828470,Co-op City,10475,Bronx,-73.829939,40.874294


In [ ]:
health_merged.shape # Display shape of merged dataset

(5795, 9)

In [ ]:
df_health_comp = df_health.copy() # Create copies of dataframes to work on reducing information loss
health_merged_comp = health_merged.copy()

In [ ]:
Comp_df = [df_health_comp, health_merged_comp]
comp_df = pd.concat(Comp_df, axis=1) # Concatenate copied dataframes
null_data = comp_df[comp_df.isnull().any(axis=1)] # Create dataframe where NaN data exists
null_data = null_data.loc[:,~null_data.columns.duplicated()].copy() # Remove non-unique column names
null_data['Neighborhood'].value_counts() # Get counts of neighborhoods with NaN data

Gravesend              44
Manhattan Terrace      36
East Flatbush          20
Brooklyn Heights       17
Van Nest               16
Allerton               14
Sunnyside              12
Prince's Bay           12
Dumbo                  12
Chelsea, Manhattan      9
Broadway Junction       9
Bronxdale               9
Hell's Kitchen          7
Pelham Parkway          6
Glendale                5
Downtown Brooklyn       5
Weeksville              5
Egbertville             4
Ocean Hill              4
Crown Heights           4
Fulton Ferry            4
Kingsbridge Heights     3
Hammels                 3
Sheepshead Bay          2
Tottenville             2
Pleasant Plains         2
Manor Heights           2
Long Island City        2
Stapleton               2
Blissville              1
Queensbridge            1
Name: Neighborhood, dtype: int64

#### There are 44 duplicate venues in Gravesend contributing to it's high count in the null_data table

In [ ]:
Gra_1 = null_data[null_data['Neighborhood'] == 'Gravesend']['Venue'] # Filter venues from neighborhood with most missing values
Gra_2 = health_merged[health_merged['Neighborhood'] == 'Gravesend']['Venue'] # Filter venues from same neighborhood prior to null data set merge
Gra_3 = Gra_1.tolist() # Append to list
Gra_4 = Gra_2.tolist()
Gra_5 = Gra_3+Gra_4 # Add both lists together
print((len(Gra_5)) - len(set(Gra_5))) # display the total length of the original dataframe from the unique length

44


#### We can see that the missing values are due to venues being duplicated in the original dataframe rather than being lost in the merge.

### Replacing inconsistent neighborhood names between the two original tables.

In [ ]:
# Replace neighborhood names in health dataset where they had originally been changed in the nyc
# neighborhood dataset

df_health['Neighborhood'] = df_health['Neighborhood'].replace('Sunnyside', 'Sunnyside, Queens')

In [ ]:
df_health['Neighborhood'] = df_health['Neighborhood'].replace('Chelsea', 'Chelsea, Manhattan')
df_health

,Venue,Category,Latitude,Longitude,Neighborhood,ZIP
0,Wakefield Eye Care,Optometrist,40.898682,-73.849352,Wakefield,10466
1,Advanced Dental Group,Dentist,40.875278,-73.828492,Co-op City,10475
2,Creating Smiles,Dentist,40.871558,-73.830285,Co-op City,10475
3,NYC Health & Hospitals,Medical Center,40.873701,-73.826771,Co-op City,10475
4,Montefiore Medical Group,Physician,40.877116,-73.828470,Co-op City,10475
...,...,...,...,...,...,...
6064,The Floating Hospital,Healthcare Clinic,40.755599,-73.945195,Queensbridge,11101
6065,The Floating Hospital,Health and Medicine,40.754157,-73.941933,Long Island City,11101
6066,Nutropia,Nutritionist,40.757598,-73.941034,Long Island City,11101
6067,Staten Island Mental Health,Mental Health Clinic,40.620457,-74.080547,Stapleton,10304


## Final Merge

In [ ]:
nyc_health_borough = df_health.merge(nyc_df, how='inner', on='Neighborhood') # Perform one final merge
nyc_health_borough.to_csv('nyc_health_borough.csv') # Save final dataframe to csv for export purposes

## Exploring the Merged Table

In [ ]:
print(nyc_health_borough.shape)
nyc_health_borough.head()

(5864, 9)


,Venue,Category,Latitude_x,Longitude_x,Neighborhood,ZIP,Borough,Longitude_y,Latitude_y
0,Wakefield Eye Care,Optometrist,40.898682,-73.849352,Wakefield,10466,Bronx,-73.847201,40.894705
1,Advanced Dental Group,Dentist,40.875278,-73.828492,Co-op City,10475,Bronx,-73.829939,40.874294
2,Creating Smiles,Dentist,40.871558,-73.830285,Co-op City,10475,Bronx,-73.829939,40.874294
3,NYC Health & Hospitals,Medical Center,40.873701,-73.826771,Co-op City,10475,Bronx,-73.829939,40.874294
4,Montefiore Medical Group,Physician,40.877116,-73.828470,Co-op City,10475,Bronx,-73.829939,40.874294


In [ ]:
nyc_health_borough_totals = nyc_health_borough.groupby('Neighborhood').size() # group by neighborhood to view total category count
nyc_health_borough_totals = nyc_health_borough_totals.reset_index() # reset index of sized counts
nyc_health_borough_totals = nyc_health_borough_totals.rename(columns={0:'Venue Counts'}) # Rename unnamed column
nyc_health_borough_totals = nyc_health_borough_totals.sort_values(by='Venue Counts', ascending=False) # Sort count column by descending
nyc_health_borough_totals

,Neighborhood,Venue Counts
39,Chinatown,174
64,East Village,131
115,Lenox Hill,124
37,"Chelsea, Manhattan",119
77,Forest Hills,113
...,...,...
95,Grymes Hill,1
170,Queensbridge,1
99,Hollis,1
3,Arlington,1


In [ ]:
 # Compare unique neighborhoods between health dataframe and original nyc neighborhood dataframe

nyc_df.shape[0], nyc_health_borough['Neighborhood'].nunique()

(306, 227)

#### There are a non-insignificant number of neighborhoods without any care facilities.

In [ ]:
# Get a list of all neighborhoods that are present in one dataframe but not the other

nyc_not_in = set(nyc_df['Neighborhood']).difference(nyc_health_borough['Neighborhood'])
nyc_not_in

{'Bay Terrace, Staten Island',
 'Bedford Park',
 'Bedford Stuyvesant',
 'Belle Harbor',
 'Bellerose',
 'Bergen Beach',
 'Bloomfield',
 'Breezy Point',
 'Broad Channel',
 'Brookville',
 'Bulls Head',
 'Butler Manor',
 'Charleston',
 'Claremont Village',
 'Clifton',
 'Concourse Village',
 'Country Club',
 'Ditmas Park',
 'Douglaston',
 'Eastchester',
 'Edgemere',
 'Edgewater Park',
 'Elm Park',
 'Eltingville',
 'Emerson Hill',
 'Erasmus',
 'Floral Park',
 'Forest Hills Gardens',
 'Graniteville',
 'Grasmere',
 'Heartland Village',
 'High  Bridge',
 'Highland Park',
 'Homecrest',
 'Hudson Yards',
 'Huguenot',
 'Hunts Point',
 'Laurelton',
 'Lighthouse Hill',
 'Little Italy',
 'Madison',
 'Malba',
 'Manhattanville',
 'Marine Park',
 'Middle Village',
 'Mill Basin',
 'Mill Island',
 'Morris Park',
 'Neponsit',
 'Noho',
 'North Riverdale',
 'Oakwood',
 'Paerdegat Basin',
 'Pelham Bay',
 'Port Ivory',
 'Port Morris',
 'Ravenswood',
 'Rego Park',
 'Remsen Village',
 'Richmond Valley',
 'Roxbury

In [ ]:
# Store difference dataframe for future use

%store nyc_not_in

Stored 'nyc_not_in' (set)


In [ ]:
print('There are {} different care facility categories.'.format(len(nyc_health_borough['Category'].unique())))

There are 57 different care facility categories.


### Getting Dummy Variables for Categories

In [ ]:
 # Get dummy variables for categorical data

nyc_dummies = pd.get_dummies(nyc_health_borough[['Category']], prefix="", prefix_sep="")

nyc_dummies['Neighborhood'] = nyc_health_borough['Neighborhood'] # Align column names

fixed_columns = [nyc_dummies.columns[-1]] + list(nyc_dummies.columns[:-1]) # Arrange Neighborhood column to be 1st

nyc_dummies = nyc_dummies[fixed_columns]

nyc_dummies.to_csv('nyc_dummies.csv', index=False) # Save to csv

In [ ]:
# Group by neighborhood to sum category counts for each neighborhood

nyc_category_counts = nyc_dummies.groupby('Neighborhood').sum().reset_index()

# Group by neighborhood to average out category frequency of each neighborhood

nyc_category_frequencies = nyc_dummies.groupby('Neighborhood').mean().reset_index()

nyc_category_counts.to_csv('nyc_category_counts.csv', index=False)
nyc_category_frequencies.to_csv('nyc_category_frequencies.csv', index=False) # Save both dummy variable tables to csv

In [ ]:
print(nyc_category_counts.shape)
nyc_category_counts.head()

(227, 58)


,Neighborhood,AIDS Resource,Acupuncture Clinic,Alternative Medicine Clinic,Ambulance Service,Anesthesiologist,Assisted Living Service,Blood Bank,Cardiologist,Chiropractor,...,Psychiatrist,Psychologist,Radiologist,Rehabilitation Center,Respiratory Doctor,Sports Medicine Clinic,Urgent Care Center,Urologist,Veterinarian,Weight Loss Center
0,Allerton,0,0,0,0,0,2,0,0,4,...,0,0,0,0,0,0,0,0,0,0
1,Annadale,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Arden Heights,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,Arlington,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Arrochar,0,1,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
nyc_category_frequencies.head()

,Neighborhood,AIDS Resource,Acupuncture Clinic,Alternative Medicine Clinic,Ambulance Service,Anesthesiologist,Assisted Living Service,Blood Bank,Cardiologist,Chiropractor,...,Psychiatrist,Psychologist,Radiologist,Rehabilitation Center,Respiratory Doctor,Sports Medicine Clinic,Urgent Care Center,Urologist,Veterinarian,Weight Loss Center
0,Allerton,0.0,0.000000,0.0,0.00,0.0,0.117647,0.0,0.0,0.235294,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
1,Annadale,0.0,0.000000,0.0,0.25,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
2,Arden Heights,0.0,0.000000,0.0,0.00,0.0,0.250000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0
3,Arlington,0.0,0.000000,0.0,0.00,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
4,Arrochar,0.0,0.142857,0.0,0.00,0.0,0.285714,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
